In [2]:
import pandas as pd
import csv
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn.utils import shuffle
import os

In [3]:
l1 = 'eng'
l2 = 'spa'

conf = 7

In [36]:
l1_freq_fn = '/mnt/permanent/home/eszti/dipterv/panlex/data/panlex/tsv/MULTI/freq/{}'.format(l1)
l2_freq_fn = '/mnt/permanent/home/eszti/dipterv/panlex/data/panlex/tsv/MULTI/freq/{}'.format(l2)

pl_fn = '/mnt/permanent/home/eszti/dipterv/panlex/data/panlex/tsv/MULTI/filter/{0}_{1}.tsv'.format(l1, l2)

train_fn = '/mnt/permanent/home/eszti/dipterv/panlex/data/panlex/tsv/MULTI/train/{0}_{1}.tsv'.format(l1, l2)
test_fn = '/mnt/permanent/home/eszti/dipterv/panlex/data/panlex/tsv/MULTI/test/{0}_{1}.tsv'.format(l1, l2)

In [22]:
# Read l1 freq info

l1_ls = []
with open(l1_freq_fn) as f:
    lines = f.read().splitlines()
    for line in lines:
        l1_ls.append(line)

In [60]:
# 2519427
l1_f = pd.DataFrame({'eng' : l1_ls})
l1_f = l1_f.drop_duplicates(subset=('eng'))
l1_f['freq'] = l1_f.index
l1_f.head()
len(l1_f.index)

,eng,freq
0,",",0
1,.,1
2,the,2
3,</s>,3
4,of,4


2519395

In [31]:
# Read l2 freq info

l2_ls = []
with open(l2_freq_fn) as f:
    lines = f.read().splitlines()
    for line in lines:
        l2_ls.append(line)

In [58]:
# 985649
l2_f = pd.DataFrame({'spa' : l2_ls})
l2_f = l2_f.drop_duplicates(subset=('spa'))
l2_f['freq'] = l2_f.index
l2_f.head()
len(l2_f.index)

,spa,freq
0,de,0
1,</s>,1
2,",",2
3,.,3
4,la,4


985649

In [61]:
# Read panlex data
pl = pd.read_csv(pl_fn, sep='\t', header=None, names = [l1, l2, 'score'])
pl['score'] = pl.score.astype(int)
pl.head()
len(pl.index)

,eng,spa,score
0,expression,expresión,9
1,rouble,rublo,9
2,sum,sum,9
3,other,otro,9
4,marka,marco,9


341045

In [62]:
# Filter conf

pl = pl[pl.score >= conf]
len(pl.index)

73968

In [63]:
# Merge l1 freq info
pl_f = pd.merge(pl, l1_f, on=[l1])
pl_f.head()
len(pl_f.index)

,eng,spa,score,freq
0,expression,expresión,9,3477
1,expression,expresión,8,3477
2,expression,palabra,7,3477
3,expression,sentencia,7,3477
4,expression,giro,7,3477


73966

In [64]:
# Merge l2 freq info
pl_ff = pd.merge(pl_f, l2_f, on=[l2])
pl_ff.head()
len(pl_ff.index)

,eng,spa,score,freq_x,freq_y
0,expression,expresión,9,3477,1965
1,expression,expresión,8,3477,1965
2,expression,expresión,7,3477,1965
3,phrase,expresión,7,3330,1965
4,term,expresión,7,534,1965


73966

In [65]:
# Create bins from l1

bin1 = l1_f.iloc[1000:5000]
bin2 = l1_f.iloc[5000:20000]
bin3 = l1_f.iloc[20000:50000]
bin4 = l1_f.iloc[50000:100000]
bin5 = l1_f.iloc[100000:200000]

In [66]:
def get_300_uq(binN, alle, N, no_dup=True):
    print('Number of words in {0}: {1}'.format(N, len(binN.index)))
    
    alle_bin = pd.merge(alle, binN, on=[l1])
    print('len alle_bin: {0}'.format(len(alle_bin.index)))
        
    if no_dup:
        alle_bin = alle_bin.drop_duplicates(subset=[l1], keep=False)
        print('len alle_bin nodup: {0}'.format(len(alle_bin.index)))
    
    alle_bin = shuffle(alle_bin)
    
    testN = alle_bin.head(300)
    print('len testN: {0}'.format(len(testN.index)))

    return testN

In [67]:
test1 = get_300_uq(bin1, pl_ff, 1)

Number of words in 1: 4000
len alle_bin: 13845
len alle_bin nodup: 529
len testN: 300


In [68]:
test2 = get_300_uq(bin2, pl_ff, 2)

Number of words in 2: 15000
len alle_bin: 22096
len alle_bin nodup: 2163
len testN: 300


In [69]:
test3 = get_300_uq(bin3, pl_ff, 3)

Number of words in 3: 30000
len alle_bin: 16499
len alle_bin nodup: 3657
len testN: 300


In [70]:
test4 = get_300_uq(bin4, pl_ff, 4)

Number of words in 4: 50000
len alle_bin: 9658
len alle_bin nodup: 3392
len testN: 300


In [71]:
test5 = get_300_uq(bin5, pl_ff, 5)

Number of words in 5: 100000
len alle_bin: 5596
len alle_bin nodup: 2715
len testN: 300


In [74]:
# Concat test

frames = [test1, test2, test3, test4, test5]
pl_test = pd.concat(frames)
pl_test.head()
len(pl_test.index)

,eng,spa,score,freq_x,freq_y,freq
13812,stones,piedras,7,4897,3192,4897
13530,khan,kan,7,2538,14462,2538
13552,genetic,genético,7,4722,11061,4722
13783,lakes,lagos,7,3061,2962,3061
12917,vehicles,vehículos,7,2274,2210,2274


1500

In [83]:
# Create train without duplicates

test_eng = pl_test[[l1]].copy()
len(test_eng.index)

pl_train = pl_ff[~pl_ff.eng.isin(set(test_eng.eng))]
len(pl_ff.index)
len(pl_train.index)

pl_train = pl_train.sort_values(['freq_x', 'freq_y'], ascending=True)
pl_train_nodup = pl_train.drop_duplicates(subset=[l1], keep=False)
len(pl_train_nodup.index)

pl_train_nodup_5K = pl_train_nodup.head(5000)
pl_train_nodup_5K.head(10)
pl_train_nodup_5K.tail(10)
len(pl_train_nodup_5K.index)

1500

73966

72466

11044

,eng,spa,score,freq_x,freq_y
26414,had,engañar,7,52,19833
27410,t,t,8,54,298
23556,made,hecho,7,61,310
27613,%,%,8,66,43
34077,used,ocasión,7,109,2123
3587,than,que,7,112,12
69113,e,e,7,168,61
35080,john,baño,7,193,6362
448,same,mismo,7,194,99
6599,could,poder,7,196,313


,eng,spa,score,freq_x,freq_y
56602,crossbar,larguero,7,44381,78142
68031,cleanly,limpiamente,7,44390,104227
44857,bullseye,diana,7,44396,4972
69116,halogen,halógeno,7,44404,75198
66158,cancerous,canceroso,7,44410,148963
59392,chronically,crónicamente,7,44411,161955
58506,morello,guindo,7,44433,132640
49890,grammarian,gramático,7,44436,39619
72775,hernia,hernia,7,44440,44799
72379,folkloric,folclórico,7,44448,21693


5000

In [82]:
# With duplicates

pl_train_nodup = pl_train
len(pl_train_nodup.index)

pl_train_nodup_5K = pl_train_nodup.head(5000)
pl_train_nodup_5K.head(10)
pl_train_nodup_5K.tail(10)
len(pl_train_nodup_5K.index)

72466

,eng,spa,score,freq_x,freq_y
18787,",",",",8,0,2
18785,",",.,8,0,3
18788,.,",",8,1,2
18786,.,.,8,1,3
27723,the,la,7,2,4
3660,the,el,7,2,6
27739,the,los,7,2,15
27734,the,las,7,2,20
27730,the,lo,7,2,34
12356,of,de,7,4,0


,eng,spa,score,freq_x,freq_y
4756,average,medio,7,749,303
58146,average,media,7,749,523
26061,average,común,7,749,669
58150,average,promedio,7,749,1865
21385,average,regular,7,749,2050
4075,average,corriente,7,749,2117
55777,average,mediocre,7,749,27773
20780,network,red,7,750,541
29697,network,emisora,7,750,5949
29696,network,interconectar,7,750,99091


5000

In [80]:
eng_te = set(pl_test[l1])
eng_tr = set(pl_train_nodup_5K[l1])
len(eng_te & eng_tr)

spa_te = set(pl_test[l2])
spa_tr = set(pl_train_nodup_5K[l2])
len(spa_te & spa_tr)

0

110

In [30]:
def save_tsv(df, fn):
    if not os.path.exists(os.path.dirname(fn)):
        os.makedirs(os.path.dirname(fn))
    pd.DataFrame.to_csv(df, columns=[l1, l2], header=False, path_or_buf=fn, sep='\t', index=False)

In [33]:
save_tsv(pl_train_nodup_5K, train_fn)
save_tsv(pl_test, test_fn)